In [6]:
import pandas as pd
import io
from minio import Minio
import os

METADATA_KEY = "x-amz-meta-type"
METADATA_VAL = "excel_almendros_cercanos"


ACCESS_ROOT = os.environ.get("PREFECT_MINIO_ACCESS_ROOT")
SECRET_ROOT = os.environ.get("PREFECT_MINIO_SECRET_ROOT")
MINIO_HOST = os.environ.get("PREFECT_MINIO_HOST")
minio_client = Minio(MINIO_HOST, access_key=ACCESS_ROOT,
                     secret_key=SECRET_ROOT, secure=False)

In [7]:
# Fetch objects and filter by metadata
objects = minio_client.list_objects(
    "landing-zone", include_user_meta=True)
# print objects name and metadata
objects_name = []
for obj in objects:
    stat = minio_client.stat_object("landing-zone", obj.object_name)
    print(stat.metadata["x-amz-meta-type"])
    

excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos


KeyError: 'x-amz-meta-type'